In [ ]:
#$ export FLASK_APP=endpoints.py
#$ flask run
# http://127.0.0.1:5000/

In [24]:
import requests
from sql_caller import engine_connector, user_call, chat_checker
conn = engine_connector()
from sqlalchemy import text
import pandas as pd
from randomuser import RandomUser
from itertools import combinations
import random
import numpy as np
from datetime import datetime

In [2]:
## Add users to DB

user_list = RandomUser.generate_users(40)

for user in user_list:
    users_dict = {'name':user.get_first_name(), 'last_name':user.get_last_name(), 'nick_name':user.get_username()}
    endpoint = "new_user?user_name={name}&user_surname={last_name}&user_nick={nick_name}".format(**users_dict)
    
    requests.get("http://127.0.0.1:5000/" + endpoint)

In [3]:
## Add chats

query = f"""SELECT user_nick FROM chat_api.users;
         """
users_list = pd.read_sql(con=conn, sql=query)['user_nick'].tolist()

selection = random.sample(users_list, 10)
chats_comb = list(combinations(selection, 2))
ten_chats = random.sample(chats_comb, 10)

for chat in ten_chats:
    chat_dict = {'sender_nick': chat[0], 'recv_nick': chat[1]}
    endpoint = "chat/create?sender_nick={sender_nick}&recv_nick={recv_nick}".format(**chat_dict)
    
    requests.get("http://127.0.0.1:5000/" + endpoint)

In [27]:
user_id_send = user_call('ticklishlion322')
user_id_recv = user_call('smallladybug598')
chat_id = chat_checker(user_id_send, user_id_recv, chat_id=True)
message = 'Hello'

if isinstance(chat_id, np.int64):
    query = f"""INSERT INTO chat_api.chat_messages (chat_id, message, message_date, message_sender_id)
                VALUES ('{chat_id}', '{message}', '{datetime.now()}', '{user_id_send}');
            """
    conn.execute(query)
else:
    print(chat_id)

IntegrityError: (MySQLdb._exceptions.IntegrityError) (1452, 'Cannot add or update a child row: a foreign key constraint fails (`chat_api`.`chat_messages`, CONSTRAINT `fk_chat_messages_users_has_chats2` FOREIGN KEY (`message_sender_id`) REFERENCES `users_has_chats` (`user_id_send`))')
[SQL: INSERT INTO chat_api.chat_messages (chat_id, message, message_date, message_sender_id)
                VALUES ('9', 'Hello', '2020-11-29 02:13:03.510241', '8');
            ]
(Background on this error at: http://sqlalche.me/e/13/gkpj)

In [21]:
#endpoint = "chat/addmessage?sender_nick=happysnake774&recv_nick=smallladybug598&message=Hey_man"
#requests.get("http://127.0.0.1:5000/" + endpoint)

query = f"""INSERT INTO chat_api.chat_messages (chat_id, message, message_date, message_sender_id)
            VALUES (2, 'Hello_its_me', '{datetime.now()}', 10);
        """
conn.execute(query)

IntegrityError: (MySQLdb._exceptions.IntegrityError) (1452, 'Cannot add or update a child row: a foreign key constraint fails (`chat_api`.`chat_messages`, CONSTRAINT `fk_chat_messages_users_has_chats3` FOREIGN KEY (`message_sender_id`) REFERENCES `users_has_chats` (`users_id_recv`))')
[SQL: INSERT INTO chat_api.chat_messages (chat_id, message, message_date, message_sender_id)
            VALUES (2, 'Hello_its_me', '2020-11-28 14:57:56.018698', 10);
        ]
(Background on this error at: http://sqlalche.me/e/13/gkpj)

In [ ]:
@app.route("/")
def hello_world():
    return {'hello':'world'}

@app.route('/name')
def te_doy_tu_nombre():
    name = request.args.get('name')
    lastname = request.args.get('lastname')
    return {"Hello": f"{name} {lastname}"}

@app.route('/mucho')
def mucho():
    args = request.args
    res = {}
    for k, v in args.items():
        res[k] = v
    return res

